In [1]:
import unicodedata
import nltk
#nltk.download('wordnet')
#nltk.download('omw-1.4')
import re
from nltk.corpus import stopwords
from nltk.tokenize.toktok import ToktokTokenizer
import pandas as pd
import numpy as np
import requests
import os
import json
from acquire_news_articles import *

In [2]:
topic = 'technology'

In [3]:
articles = scrape_one_page(topic)

In [4]:
articles = pd.DataFrame(articles)

In [5]:
articles.head()

,category,title,content
0,technology,Twitter down for thousands of users worldwide,Microblogging platform Twitter suffered an out...
1,technology,"Bill Gates meets Ratan Tata, N Chandrasekaran;...",Microsoft Co-founder Bill Gates met with Tata ...
2,technology,We can score a century for progress: Gates on ...,Microsoft Co-founder Bill Gates shared a messa...
3,technology,Woman loses ₹8 lakh after being locked out of ...,A woman in the US has said she was locked out ...
4,technology,Meta trying to extinguish Cambridge Analytica ...,New Mexico state attorneys have accused Meta o...


Define a function named basic_clean. It should take in a string and apply some basic text cleaning to it:

Lowercase everything
Normalize unicode characters
Replace anything that is not a letter, number, whitespace or a single quote.

In [6]:
input_string = ""

In [7]:
def basic_clean(input_string):
    """
    This function takes in a string and applies basic cleaning to it.
    """
    #Changes all characters to their lower case.
    input_string = input_string.lower()
    
    input_string = unicodedata.normalize('NFKD', input_string).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    
    #Removes special characters
    input_string = re.sub(r"[^a-z0-9'\s]", '', input_string)

    return input_string

Trying it in a dataframe. Unsure how to get it to work right:

In [8]:
def cleaning_panda_strings(df):
    """ This function does the cleaning operatins on every column in a dataframe."""
    
    for col in df:
        df[col] = df[col].apply(basic_clean)
    return df

In [9]:
new_cats = []
for string in articles.category:
    string = basic_clean(string)
    new_cats.append(string)

In [10]:
new_titles = []
for string in articles.title:
    string = basic_clean(string)
    new_titles.append(string)

In [11]:
new_content = []
for string in articles.content:
    string = basic_clean(string)
    new_content.append(string)

In [12]:
cleaned_dict = {'category':new_cats,
                'title': new_titles,
                'content': new_content}

In [13]:
cleaned_df = pd.DataFrame(cleaned_dict)

In [14]:
cleaned_df.head()

,category,title,content
0,technology,twitter down for thousands of users worldwide,microblogging platform twitter suffered an out...
1,technology,bill gates meets ratan tata n chandrasekaran p...,microsoft cofounder bill gates met with tata s...
2,technology,we can score a century for progress gates on m...,microsoft cofounder bill gates shared a messag...
3,technology,woman loses 8 lakh after being locked out of a...,a woman in the us has said she was locked out ...
4,technology,meta trying to extinguish cambridge analytica ...,new mexico state attorneys have accused meta o...


In [15]:
test_df = cleaning_panda_strings(articles)

In [16]:
test_df.head()

,category,title,content
0,technology,twitter down for thousands of users worldwide,microblogging platform twitter suffered an out...
1,technology,bill gates meets ratan tata n chandrasekaran p...,microsoft cofounder bill gates met with tata s...
2,technology,we can score a century for progress gates on m...,microsoft cofounder bill gates shared a messag...
3,technology,woman loses 8 lakh after being locked out of a...,a woman in the us has said she was locked out ...
4,technology,meta trying to extinguish cambridge analytica ...,new mexico state attorneys have accused meta o...


Define a function named tokenize. It should take in a string and tokenize all the words in the string.

tokenizer = nltk.tokenize.ToktokTokenizer()

new_category_tokens = []
for article2 in cleaned_df.category:
    article2 = tokenizer.tokenize(article2, return_str=True)
    new_category_tokens.append(article2)

new_title_tokens = []
for article0 in cleaned_df.title:
    article0 = tokenizer.tokenize(article0, return_str=True)
    new_title_tokens.append(article0)

new_content_tokens = []
for article1 in cleaned_df.content:
    article1 = tokenizer.tokenize(article1, return_str=True)
    new_content_tokens.append(article1)

tolken_dict = {'category':new_category_tokens,
                'title': new_title_tokens,
                'content': new_content_tokens}

tolken_df = pd.DataFrame(tolken_dict)

tolken_df.head()

In [17]:
def tokenized(input_string):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    return tokenizer.tokenize(input_string, return_str=True)

In [18]:
def panda_tokens(df):
    for col in df:
        df[col] = df[col].apply(tokenized)
    return df

In [19]:
tokenized_df = panda_tokens(test_df)

Define a function named stem. It should accept some text and return the text after applying stemming to all the words.

In [20]:
def stem(input_string):
    """
    Takes a string and reverts each word to its base stem.
    Please do not stem and lemmatize the same string.
    """
    #Create stemming object
    ps = nltk.porter.PorterStemmer()

    #List comprehension that splits every word to find the stem of it.
    stems = [ps.stem(word) for word in input_string.split()]

    #Joins words back together to form a shadow of the origional
    stemmed_string = ' '.join(stems)
    
    return stemmed_string

In [21]:
def stemming_pandas(df):
    """
    Does stemming on a panda's DataFrame
    Please do not stem and lemmatize the same string.
    """
    
    #Applies the stemming() function to each column
    for col in df:
        df[col] = df[col].apply(stem)
    return df

In [22]:
stemmed_df = stemming_pandas(tokenized_df)

In [23]:
stemmed_df.head()

,category,title,content
0,technolog,twitter down for thousand of user worldwid,microblog platform twitter suffer an outag on ...
1,technolog,bill gate meet ratan tata n chandrasekaran pic...,microsoft cofound bill gate met with tata son ...
2,technolog,we can score a centuri for progress gate on me...,microsoft cofound bill gate share a messag on ...
3,technolog,woman lose 8 lakh after be lock out of appl ac...,a woman in the us ha said she wa lock out of h...
4,technolog,meta tri to extinguish cambridg analytica clai...,new mexico state attorney have accus meta of w...


Define a function named lemmatize. It should accept some text and return the text after applying lemmatization to each word.

In [24]:
def lemmatize(input_string):
    """
    Takes an input string and lemmatizes it.
    Please do not stem and lemmatize the same string.
    """
    #Creates the lemmatizer object
    wnl = nltk.stem.WordNetLemmatizer()
    
    #Makes lemmatade
    lemmas = [wnl.lemmatize(word) for word in input_string.split()]
    lemmatized_string = ' '.join(lemmas)
    
    return lemmatized_string

In [25]:
def lemm_pandas(df):
    """
    Lemmatize a panda's DataFrame
    Please do not stem and lemmatize the same string.
    """
    
    #Applies the lemmatize() function to each column
    for col in df:
        df[col] = df[col].apply(lemmatize)
    return df

In [26]:
lemm_df = lemm_pandas(tokenized_df) #Made sure to use a non-stemmed df

In [27]:
lemm_df.head()

,category,title,content
0,technolog,twitter down for thousand of user worldwid,microblog platform twitter suffer an outag on ...
1,technolog,bill gate meet ratan tata n chandrasekaran pic...,microsoft cofound bill gate met with tata son ...
2,technolog,we can score a centuri for progress gate on me...,microsoft cofound bill gate share a messag on ...
3,technolog,woman lose 8 lakh after be lock out of appl ac...,a woman in the u ha said she wa lock out of he...
4,technolog,meta tri to extinguish cambridg analytica clai...,new mexico state attorney have accus meta of w...


Define a function named remove_stopwords. It should accept some text and return the text after removing all the stopwords.

In [28]:
def remove_stopwords(input_string, extra_words=None, exclude_words=None):
    """
    This function takes an input and removes stopwords. You can add or remove words witht the extra_words or exclude_words args.
    """
    
    stopword_list = stopwords.words('english')
    if extra_words != None:
        extra_words.extend(extra_words)
    if exclude_words != None:
        if type(extra_words) == str:
            stopword_list.remove(extra_words)
        if type(extra_words) == list:
            for word in extra_words:
                stopword_list.remove(word)
    words = input_string.split()
    filtered_words = [w for w in words if w not in stopword_list]
    article_without_stopwords = ' '.join(filtered_words)

    return article_without_stopwords

In [29]:
def pandas_stopwords(df, extra_words=None, exclude_words=None):
    """
    This function takes an DataFrame input and removes stopwords from each column.
    You can add or remove words witht the extra_words or exclude_words args.
    """
    for col in df:
        df[col] = df[col].apply(remove_stopwords, extra_words=extra_words, exclude_words=exclude_words)
    return df

In [30]:
stopwordsless_lemm_df = pandas_stopwords(lemm_df)
stopwordsless_stem_df = pandas_stopwords(stemmed_df)

In [31]:
stopwordsless_lemm_df.head()

,category,title,content
0,technolog,twitter thousand user worldwid,microblog platform twitter suffer outag wednes...
1,technolog,bill gate meet ratan tata n chandrasekaran pic...,microsoft cofound bill gate met tata son chair...
2,technolog,score centuri progress gate meet tendulkar,microsoft cofound bill gate share messag twitt...
3,technolog,woman lose 8 lakh lock appl account u,woman u ha said wa lock appl account thief sto...
4,technolog,meta tri extinguish cambridg analytica claim u...,new mexico state attorney accus meta wrong cla...


In [32]:
stopwordsless_stem_df.head()

,category,title,content
0,technolog,twitter thousand user worldwid,microblog platform twitter suffer outag wednes...
1,technolog,bill gate meet ratan tata n chandrasekaran pic...,microsoft cofound bill gate met tata son chair...
2,technolog,score centuri progress gate meet tendulkar,microsoft cofound bill gate share messag twitt...
3,technolog,woman lose 8 lakh lock appl account u,woman u ha said wa lock appl account thief sto...
4,technolog,meta tri extinguish cambridg analytica claim u...,new mexico state attorney accus meta wrong cla...


This function should define two optional parameters, extra_words and exclude_words. These parameters should define any additional stop words to include, and any words that we don't want to remove.

In [33]:
print("Done")

Done


Use your data from the acquire to produce a dataframe of the news articles. Name the dataframe news_df.

In [34]:
news_df = stopwordsless_lemm_df.copy

Make another dataframe for the Codeup blog posts. Name the dataframe codeup_df.

In [35]:
headers = {'User-Agent': 'Codeup Data Science'}


In [36]:
#Scrape blog homepage for links
url = 'https://codeup.com/blog/'

response = get(url, headers=headers)

soup = BeautifulSoup(response.content, 'html.parser')

more_links = soup.find_all('a', class_='more-link')

more_links

[<a class="more-link" href="https://codeup.com/codeup-news/panelist-spotlight-4/">read more</a>,
 <a class="more-link" href="https://codeup.com/events/black-excellence-in-tech-panelist-spotlight-stephanie-jones/">read more</a>,
 <a class="more-link" href="https://codeup.com/events/black-excellence-in-tech-panelist-spotlight-james-cooper/">read more</a>,
 <a class="more-link" href="https://codeup.com/events/black-excellence-in-tech-panelist-spotlight/">read more</a>,
 <a class="more-link" href="https://codeup.com/tips-for-prospective-students/coding-bootcamp-or-self-learning/">read more</a>,
 <a class="more-link" href="https://codeup.com/codeup-news/codeup-best-bootcamps/">read more</a>]

In [37]:
#Extract the links into something I can use
links_list = [link['href'] for link in more_links]

links_list

['https://codeup.com/codeup-news/panelist-spotlight-4/',
 'https://codeup.com/events/black-excellence-in-tech-panelist-spotlight-stephanie-jones/',
 'https://codeup.com/events/black-excellence-in-tech-panelist-spotlight-james-cooper/',
 'https://codeup.com/events/black-excellence-in-tech-panelist-spotlight/',
 'https://codeup.com/tips-for-prospective-students/coding-bootcamp-or-self-learning/',
 'https://codeup.com/codeup-news/codeup-best-bootcamps/']

In [38]:
codeup_text = get_blog_articles(links_list)

In [40]:
codeup_df = pd.DataFrame(codeup_text)

In [41]:
clean_codeup_df = cleaning_panda_strings(codeup_df)

For each dataframe, produce the following columns:

title to hold the title
original to hold the original article/post content
clean to hold the normalized and tokenized original with the stopwords removed.
stemmed to hold the stemmed version of the cleaned data.
lemmatized to hold the lemmatized version of the cleaned data.

Ask yourself:

If your corpus is 493KB, would you prefer to use stemmed or lemmatized text?
- lemmatized
If your corpus is 25MB, would you prefer to use stemmed or lemmatized text?
- lemmatized
If your corpus is 200TB of text and you're charged by the megabyte for your hosted computational resources, would you prefer to use stemmed or lemmatized text?
- My money = stemmed, someone else's money = lemmatized.